In [1]:
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import SGD

def acc(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class basic_cnn(pl.LightningModule):
    def __init__(self):
        super(basic_cnn,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv3 = nn.Conv2d(6, 12, 3)
        self.max3 = nn.MaxPool2d(2)
        self.fc2 = nn.Flatten()
        self.d1 = nn.Linear(46128,101)
        self.lr = 0.01
        self.loss = nn.CrossEntropyLoss()
        
           
    def forward(self, x):
        batch_size, _, _, _ = x.size()
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.max3(x))
        x = self.fc2(x)
        x = F.relu(self.d1(x))
        return x
    
    def configure_optimizers(self):
        # Define Optimizer Here
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat,y)
        self.log("train_loss", loss)
        return loss
        
    def validation_step(self, valid_batch, batch_idx):
        x, y = valid_batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat,y)
        
        pred = acc(y_hat, y)
        self.log("val_accuracy", pred[0])
        self.log("val_loss", loss)
        return loss
    
    def test_step(self, test_batch, batch_idx):
        l_test=[]
        
        x, y = test_batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat,y)

        l_test.append(loss)
        
        pred = acc(y_hat, y)

        
        self.log("test_accuracy", pred[0])
        self.log("test_loss", loss)
        return l_test[-1]


In [2]:
import pytorch_lightning as pl
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Food101
import torch
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

class Food101(pl.LightningDataModule):
    def prepare_data(self):
        transform=transforms.Compose([
            transforms.RandomResizedCrop(128),
            transforms.RandomCrop(128),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5],
                                 [0.229, 0.224, 0.225])])
      
        self.train_data = datasets.Food101('C/Users/jasti/Downloads/data', split="train", download=True, transform=transform)
        self.val_data = datasets.Food101('C/Users/jasti/Downloads/data', split="test", download=True, transform=transform)
        self.test_data = datasets.Food101('C/Users/jasti/Downloads/data', split="test", download=True, transform=transform)
        print('[INFO] data loaded successfully')

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size= 32, shuffle=True, num_workers=8)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size= 32, shuffle=True, num_workers=8)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size= 32, shuffle=True, num_workers=8)


/usr/local/lib/python3.8/dist-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/usr/local/lib/python3.8/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/usr/local/lib/python3.8/dist-packages/pl_bolts/models/self_supervised

In [3]:
model = basic_cnn()
Food101 = Food101()
trainer = pl.Trainer(accelerator ="cpu", max_epochs=5)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model, Food101)

  0%|          | 0/4996278331 [00:00<?, ?it/s]

Extracting C/Users/jasti/Downloads/data/food-101.tar.gz to C/Users/jasti/Downloads/data


INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | conv1 | Conv2d           | 168   
1 | conv3 | Conv2d           | 660   
2 | max3  | MaxPool2d        | 0     
3 | fc2   | Flatten          | 0     
4 | d1    | Linear           | 4.7 M 
5 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params
18.639    Total estimated model params size (MB)


[INFO] data loaded successfully


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [5]:
trainer.test(model, Food101)

[INFO] data loaded successfully


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          6.899009704589844
        test_loss            4.37167501449585
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 6.899009704589844, 'test_loss': 4.37167501449585}]

In [6]:
trainer.validate(model, Food101)

[INFO] data loaded successfully


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy           6.887128829956055
        val_loss            4.3728227615356445
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_accuracy': 6.887128829956055, 'val_loss': 4.3728227615356445}]